In [1]:
import os
import pymysql
pymysql.install_as_MySQLdb()
from dotenv import load_dotenv

## Connect DB

In [2]:
load_dotenv()
ENDPOINT = os.environ.get("dev_db_endpoint")
USER = os.environ.get("dev_db_user")
PASSWORD = os.environ.get("dev_db_password")

In [3]:
conn = pymysql.connect(
    host=ENDPOINT,
    user=USER,
    password=PASSWORD,
    database="aresa",
    port=3306
)

In [4]:
cur = conn.cursor()

In [5]:
from sqlalchemy import create_engine
import sqlalchemy

In [6]:
engine = create_engine(f"mysql+mysqldb://{USER}:{PASSWORD}@{ENDPOINT}:3306/aresa")
# http://i5on9i.blogspot.com/2020/05/no-module-named-mysqldb.html

## Insert Data
- 데이터 출처: https://www.data.go.kr/data/15063424/fileData.do?recommendDataYn=Y

In [75]:
import pandas as pd

In [73]:
mois_bjd_code_info = pd.read_csv("data/국토교통부_전국 법정동_20211217.csv", encoding="cp949")

In [74]:
mois_bjd_code_info

,법정동코드,시도명,시군구명,읍면동명,리명,순위,생성일자,삭제일자,과거법정동코드
0,4223033031,강원도,삼척시,노곡면,상군천리,11.0,1995-01-01,NaN,NaN
1,4223033032,강원도,삼척시,노곡면,둔달리,12.0,1995-01-01,NaN,NaN
2,4223033033,강원도,삼척시,노곡면,주지리,13.0,1995-01-01,NaN,NaN
3,4223033034,강원도,삼척시,노곡면,하마읍리,14.0,1995-01-01,NaN,NaN
4,4223033035,강원도,삼척시,노곡면,중마읍리,15.0,1995-01-01,NaN,NaN
...,...,...,...,...,...,...,...,...,...
46284,4913090400,제주도,서귀포시,천지동,NaN,0.0,1988-04-23,1988-04-23,NaN
46285,4913090500,제주도,서귀포시,효돈동,NaN,0.0,1988-04-23,1988-04-23,NaN
46286,4913090600,제주도,서귀포시,영천동,NaN,0.0,1988-04-23,1988-04-23,NaN
46287,4913090700,제주도,서귀포시,대륜동,NaN,0.0,1988-04-23,1988-04-23,NaN


In [76]:
code_split_list = []
for s in mois_bjd_code_info.법정동코드:
    s = str(s)
    code_split_list.append([s[:5], s[5:]])

In [77]:
mois_bjd_code_info[["법정동_시군구코드", "법정동_읍면동코드"]] = code_split_list

In [78]:
mois_bjd_code_info

,법정동코드,시도명,시군구명,읍면동명,리명,순위,생성일자,삭제일자,과거법정동코드,법정동_시군구코드,법정동_읍면동코드
0,4223033031,강원도,삼척시,노곡면,상군천리,11.0,1995-01-01,NaN,NaN,42230,33031
1,4223033032,강원도,삼척시,노곡면,둔달리,12.0,1995-01-01,NaN,NaN,42230,33032
2,4223033033,강원도,삼척시,노곡면,주지리,13.0,1995-01-01,NaN,NaN,42230,33033
3,4223033034,강원도,삼척시,노곡면,하마읍리,14.0,1995-01-01,NaN,NaN,42230,33034
4,4223033035,강원도,삼척시,노곡면,중마읍리,15.0,1995-01-01,NaN,NaN,42230,33035
...,...,...,...,...,...,...,...,...,...,...,...
46284,4913090400,제주도,서귀포시,천지동,NaN,0.0,1988-04-23,1988-04-23,NaN,49130,90400
46285,4913090500,제주도,서귀포시,효돈동,NaN,0.0,1988-04-23,1988-04-23,NaN,49130,90500
46286,4913090600,제주도,서귀포시,영천동,NaN,0.0,1988-04-23,1988-04-23,NaN,49130,90600
46287,4913090700,제주도,서귀포시,대륜동,NaN,0.0,1988-04-23,1988-04-23,NaN,49130,90700


In [79]:
mois_bjd_code_info.to_sql('mois_bjd_code_info', engine, schema="aresa", index=False, if_exists='replace', chunksize=1000)

46289

In [64]:
import pandas as pd
import more_itertools as mit  # https://stackoverflow.com/questions/3438756/some-built-in-to-pad-a-list-in-python
# list(mit.padded(mois_bjd_code_info.법정동명[2].split(), None, 4))

In [65]:
mois_bjd_code_info = pd.read_csv("data/법정동코드_전체자료.txt", encoding="cp949", sep="\t")

In [66]:
mois_bjd_code_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46340 entries, 0 to 46339
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   법정동코드   46340 non-null  int64 
 1   법정동명    46340 non-null  object
 2   폐지여부    46340 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


In [67]:
as1_to_as4_list = []
code_list = []

In [68]:
for s in mois_bjd_code_info.법정동명:
    data = list(mit.padded(s.split(), None, 4))
    as1_to_as4_list.append(data)

In [69]:
for s in mois_bjd_code_info.법정동코드:
    s = str(s)
    code_list.append([s[:5], s[5:]])

In [62]:
mois_bjd_code_info[["시도명", "시군구명", "읍면동명", "리명"]] = pd.DataFrame(as1_to_as4_list)

ValueError: Columns must be same length as key

In [71]:
pd.DataFrame(as1_to_as4_list)

,0,1,2,3,4
0,서울특별시,None,None,None,None
1,서울특별시,종로구,None,None,None
2,서울특별시,종로구,청운동,None,None
3,서울특별시,종로구,신교동,None,None
4,서울특별시,종로구,궁정동,None,None
...,...,...,...,...,...
46335,제주특별자치도,서귀포시,표선면,하천리,None
46336,제주특별자치도,서귀포시,표선면,성읍리,None
46337,제주특별자치도,서귀포시,표선면,가시리,None
46338,제주특별자치도,서귀포시,표선면,세화리,None


In [54]:
mois_bjd_code_info[["시군구코드", "법정동읍면동코드"]] = code_alist

In [17]:
mois_bjd_code_info.to_sql('mois_bjd_code_info', engine, schema="aresa", index=False, if_exists='replace', chunksize=1000)

46340

In [13]:
# bjdong_cd
# sigungu_cd
# bjd_umd_ri_cd
# sido_nm
# sigungu_nm
# umd_nm
# ri_nm